## Load Packages

In [1]:
using Dates, LinearAlgebra, Distributions

include("printmat.jl")

printlnPs (generic function with 2 methods)

# Log Utility

## Log Utility and One Risky Asset (and a riskfree)

In [2]:
rf   = 0.03                       
σ²   = 100/10000
μe1A = 1/100         #mean excess returns, state A
μe1B = 0.5/100       #mean excess returns, state B

vA = (μe1A + σ²/2)/σ²
vB = (μe1B + σ²/2)/σ²

println("Weight on risky asset and riskfree in the two states")
printmat([vA vB;1-sum(vA) 1-sum(vB)],10,2)

Weight on risky asset and riskfree in the two states
      1.50      1.00
     -0.50      0.00



## Log Utility and Many Assets (3 risky and one riskfree)

In [3]:
function OptPortLogUtil(Σ,μe,rf)     #Campbell&Viceira, log return is N(μe+rf,Σ)
  v     = inv(Σ)*(μe+diag(Σ)/2)      #warning: (μe,Σ) are for logs
  Erp   = rf + v'μe + v'diag(Σ)/2 - v'Σ*v/2
  Varrp = v'Σ*v
  return v, Erp, Varrp
end

OptPortLogUtil (generic function with 1 method)

In [4]:
Σ  = [166  34  58;
       34  64   4;
       58   4 100]/10000
μeA = [2.0, 1.0, 0.5]/100       #mean excess returns, state A
μeB = [2.0, 0.0, 0.5]/100       #mean excess returns, state B

println()

In [5]:
vA, = OptPortLogUtil(Σ,μeA,rf)
vB, = OptPortLogUtil(Σ,μeB,rf)

println("weights on risky and riskfree assets in state A and B")
printmat([vA vB;1-sum(vA) 1-sum(vB)],10,2)

weights on risky and riskfree assets in state A and B
      1.38      1.81
      1.32     -0.46
      0.14     -0.03
     -1.85     -0.32



# CRRA Utility

In [6]:
γ = 3

vA = (μe1A + σ²/2)/(σ²*γ)
vB = (μe1B + σ²/2)/(σ²*γ)

println("Weight on risky asset and riskfree in the two states")
printmat([vA vB;1-sum(vA) 1-sum(vB)],10,2)

Weight on risky asset and riskfree in the two states
      0.50      0.33
      0.50      0.67



In [7]:
function OptPortCRRA(Σ,μe,rf,γ)         #Campbell&Viceira, log return is N(μe+rf,Σ)
  v     = inv(Σ)*(μe+diag(Σ)/2)/γ      #warning: (μe,Σ) are for logs
  Erp   = rf + v'μe + v'diag(Σ)/2 - v'Σ*v/2
  Varrp = v'Σ*v
  return v, Erp, Varrp
end

OptPortCRRA (generic function with 1 method)

In [8]:
γ = 3

vA, = OptPortCRRA(Σ,μeA,rf,γ)
vB, = OptPortCRRA(Σ,μeB,rf,γ)

println("weights on risky and riskfree assets in state A and B")
printmat([vA vB;1-sum(vA) 1-sum(vB)],10,2)

weights on risky and riskfree assets in state A and B
      0.46      0.60
      0.44     -0.15
      0.05     -0.01
      0.05      0.56



# Intertemporal Hedging

## A Single Risky Asset 

and a riskfree

In [9]:
#for a single risky asset
function CRRAPortOpt1(Σ_u,Σ_η,Σ_uη,a,ϕ,γ,z)
  v_myop  = inv(Σ_u)*(a + z + Σ_u/2)/γ
  Σ       = 2*Σ_u + Σ_η + 2*Σ_uη
  v_noreb = inv(Σ)*(2*a + (I+ϕ)*z + Σ/2)/γ
  Ev1     = inv(Σ_u)*(a + ϕ*z + Σ_u/2)/γ
  v_rebal = inv(Σ_u)*(a + z + Σ_u/2 + (1-γ)*Σ_uη*Ev1)/γ
  return v_myop, v_noreb, v_rebal
end

CRRAPortOpt1 (generic function with 1 method)

In [10]:
(γ,Σ_u,Σ_η,a,ϕ) = (3,100/10000,75/10000,0.75/100,0.9)     #parameters in example with 1 risky asset

Σ_uηL  = -0.5*sqrt(Σ_u*Σ_η)     #corr(u,η)=-0.5
Σ_uηH  =  0.5*sqrt(Σ_u*Σ_η)     #corr(u,η)=0.5

(v_myopL,v_norebL,v_rebalL) = CRRAPortOpt1(Σ_u,Σ_η,Σ_uηL,a,ϕ,γ,0)
(v_myopH,v_norebH,v_rebalH) = CRRAPortOpt1(Σ_u,Σ_η,Σ_uηH,a,ϕ,γ,0)

println("Portfolio weight of risk asset: myopic and 2-period (rebalancing)\n")
println("corr(u,η)=-0.5")
printmat([v_myopL v_rebalL])
println("corr(u,η)= 0.5")
printmat([v_myopH v_rebalH])

Portfolio weight of risk asset: myopic and 2-period (rebalancing)

corr(u,η)=-0.5
     0.417     0.537

corr(u,η)= 0.5
     0.417     0.296



## Three Risky Assets

and a riskfree

In [11]:
function CRRAPortOpt(Σ_u,Σ_η,Σ_uη,a,ϕ,γ,z)
  v_myop  = inv(Σ_u)*(a + z + diag(Σ_u)/2)/γ
  Σ       = 2*Σ_u + Σ_η + 2*Σ_uη
  v_noreb = inv(Σ)*(2*a + (I+ϕ)*z + diag(Σ)/2)/γ
  Ev1     = inv(Σ_u)*(a + ϕ*z + diag(Σ_u)/2)/γ
  v_rebal = inv(Σ_u)*(a + z + diag(Σ_u)/2 + (1-γ)*Σ_uη*Ev1)/γ
  return v_myop, v_noreb, v_rebal
end

CRRAPortOpt (generic function with 1 method)

In [12]:
γ = 3

Σ_u    = [166  34  58;
           34  64   4;
           58   4 100]/10000
Σ_η    = [0  0   0;
          0  100 0;
          0  0   0]/10000       #only asset 2 has dynamics in expected returns
a      = [2,0.5,0.5]/100
ϕ      = [0  0   0;             #doesn't matter for 2-period problem when z_t=0
          0  0.9 0;
          0  0   0]

Σ_uηL =  [0  0   0;            #Cov(u,η)<0 for asset 2
          0  -40  0;
          0  0    0]/10000       
Σ_uηH =  [0  0   0;            #Cov(u,η)>0 for asset 2
          0  40   0;
          0  0    0]/10000       


println("Σ_u*10000")
printmat(Σ_u*10000)

println("Σ_η*10000")
printmat(Σ_η*10000)

println("a*100")
printmat(a*100)


println("\nTwo cases: negative and positive Cov(u,η) for asset 2")
printmat(Σ_uηL*10000)
printmat(Σ_uηH*10000)

Σ_u*10000
   166.000    34.000    58.000
    34.000    64.000     4.000
    58.000     4.000   100.000

Σ_η*10000
     0.000     0.000     0.000
     0.000   100.000     0.000
     0.000     0.000     0.000

a*100
     2.000
     0.500
     0.500


Two cases: negative and positive Cov(u,η) for asset 2
     0.000     0.000     0.000
     0.000   -40.000     0.000
     0.000     0.000     0.000

     0.000     0.000     0.000
     0.000    40.000     0.000
     0.000     0.000     0.000



In [13]:
(v_myopL,v_norebL,v_rebalL) = CRRAPortOpt(Σ_u,Σ_η,Σ_uηL,a,ϕ,γ,zeros(3))     #different Cov(u,η)
(v_myopH,v_norebH,v_rebalH) = CRRAPortOpt(Σ_u,Σ_η,Σ_uηH,a,ϕ,γ,zeros(3))

println("Portfolio weights of 3 risky asset: myopic and 2-period (rebalancing)\n")
println("cov(u,η) < 0")
printmat([v_myopL v_rebalL])
println("corr(u,η) > 0")
printmat([v_myopH v_rebalH])

Portfolio weights of 3 risky asset: myopic and 2-period (rebalancing)

cov(u,η) < 0
     0.532     0.516
     0.143     0.211
     0.019     0.026

corr(u,η) > 0
     0.532     0.549
     0.143     0.075
     0.019     0.012

